# 語音轉文字

In [ ]:
# import flask related
from flask import Flask, request, abort
# import linebot related
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, AudioMessage,
    LocationSendMessage, ImageSendMessage, StickerSendMessage
)








# create flask server
app = Flask(__name__)
# your linebot message API - Channel access token (from LINE Developer)
line_bot_api = LineBotApi('TID6M67HYlTYOEDZAtRF82nPS6qQb3eu+NKv5JWE35iq5O4cuYX3ad6+986rxEfQMDCIj2npOafm2BGITvUYB8UZWp+2dWDify7IyXoGjOsJWzJnlK69ATCGIFFe2EdVoPnUCI5ONasZ96h9QEuGHQdB04t89/1O/w1cDnyilFU=')
# your linebot message API - Channel secret
handler = WebhookHandler('e8de57a5bb8575f4440be45d44399d75')

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
    return 'OK'

# handle msg
import os
import speech_recognition as sr
'''
def transcribe(wav_path):
    '''
    Speech to Text by Google free API
    language: en-US, zh-TW
    '''
    
    r = sr.Recognizer()
    with sr.AudioFile(wav_path) as source:
        audio = r.record(source)
    try:
        return r.recognize_google(audio, language="zh-TW")
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
    return None
'''

import azure.cognitiveservices.speech as speechsdk

def transcribe(wav_path):
    # Creates an instance of a speech config with specified subscription key and service region.
    # Replace with your own subscription key and region identifier from here: https://aka.ms/speech/sdkregion
    speech_key, service_region = "c614b5c49e13466fa6cad11e10c0bd9c", "westus2"
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)

    # Creates an audio configuration that points to an audio file.
    # Replace with your own audio filename.
    wav_path = "narration.wav"
    audio_input = speechsdk.audio.AudioConfig(filename=wav_path)

    # Creates a recognizer with the given settings
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)

    print("Recognizing first result...")

    # Starts speech recognition, and returns after a single utterance is recognized. The end of a
    # single utterance is determined by listening for silence at the end or until a maximum of 15
    # seconds of audio is processed.  The task returns the recognition text as result. 
    # Note: Since recognize_once() returns only a single utterance, it is suitable only for single
    # shot recognition like command or query. 
    # For long-running multi-utterance recognition, use start_continuous_recognition() instead.
    result = speech_recognizer.recognize_once()
    return result.text

    # Checks result.
   # if result.reason == speechsdk.ResultReason.RecognizedSpeech:
   #     print("Recognized: {}".format(result.text))
   # elif result.reason == speechsdk.ResultReason.NoMatch:
   #     print("No speech could be recognized: {}".format(result.no_match_details))
   # elif result.reason == speechsdk.ResultReason.Canceled:
   #     cancellation_details = result.cancellation_details
   #     print("Speech Recognition canceled: {}".format(cancellation_details.reason))
   #     if cancellation_details.reason == speechsdk.CancellationReason.Error:
   #         print("Error details: {}".format(cancellation_details.error_details))


@handler.add(MessageEvent, message=AudioMessage)
def handle_audio(event):

    name_mp3 = 'recording.mp3'
    name_wav = 'recording.wav'
    message_content = line_bot_api.get_message_content(event.message.id)
    
    with open(name_mp3, 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)
    
    os.system('ffmpeg -y -i ' + name_mp3 + ' ' + name_wav + ' -loglevel quiet')
    
    
    text = transcribe(name_wav)
    print('Transcribe:', text)
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text = text))













'''
    
@handler.add(MessageEvent, message=AudioMessage)
def handle_audio(event):

    name_mp3 = 'recording.mp3'
    name_wav = 'recording.wav'
    message_content = line_bot_api.get_message_content(event.message.id)
    
    with open(name_mp3, 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)
    
    os.system('ffmpeg -y -i ' + name_mp3 + ' ' + name_wav + ' -loglevel quiet')
    text = transcribe(name_wav)
    print('Transcribe:', text)
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text = text))
'''
# run app
if __name__ == "__main__":
    app.run(host='127.0.0.1', port=12345)

# 照片轉文字

In [ ]:
# import flask related
from flask import Flask, request, abort
# import linebot related
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
    LocationSendMessage, ImageSendMessage, StickerSendMessage
)

# create flask server
app = Flask(__name__)
# your linebot message API - Channel access token (from LINE Developer)
line_bot_api = LineBotApi('TID6M67HYlTYOEDZAtRF82nPS6qQb3eu+NKv5JWE35iq5O4cuYX3ad6+986rxEfQMDCIj2npOafm2BGITvUYB8UZWp+2dWDify7IyXoGjOsJWzJnlK69ATCGIFFe2EdVoPnUCI5ONasZ96h9QEuGHQdB04t89/1O/w1cDnyilFU=')
# your linebot message API - Channel secret
handler = WebhookHandler('e8de57a5bb8575f4440be45d44399d75')


# Set API key.
subscription_key = 'e5847658ef394d0b960f00795989a546'
# Set endpoint.
endpoint = 'https://modulecomputervision.cognitiveservices.azure.com/'
# Call API
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        print('receive msg')
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
    return 'OK'



    


# handle msg
@handler.add(MessageEvent, message=TextMessage)
def handle_image(event):
  
    # get user info & message
    user_id = event.source.user_id
    msg = event.message.text
    user_name = line_bot_api.get_profile(user_id).display_name
    
    # push text_msg
    line_bot_api.push_message(user_id,
                              TextSendMessage(text = user_name+'您好^^'))
    
    # get msg details
    print('msg from [', user_name, '](', user_id, ') : ', msg)
    
    line_bot_api.reply_message(event.reply_token, 
                               [TextSendMessage(text = '請上傳您的筆記照片'),
                                StickerSendMessage(package_id='4',sticker_id='626')])
    
    message_content = line_bot_api.get_message_content(message_id)

    with open(file_path, 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)
      
    
    



    '''
    Batch Read File, recognize printed text - remote
    This example will extract printed text in an image, then print results, line by line.
    This API call can also recognize handwriting (not shown).
    '''

    # Get an image with printed handwritten
    #remote_image_printed_text_url = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/printed_handwritten.pdf"

    # Call API with URL and raw response (allows you to get the operation location)
    #recognize_printed_results = computervision_client.batch_read_file(remote_image_printed_text_url,  raw=True)

    recognize_printed_results = computervision_client.batch_read_file(fd)
    
    
    
    # Get the operation location (URL with an ID at the end) from the response
    operation_location_remote = recognize_printed_results.headers["Operation-Location"]
    # Grab the ID from the URL
    operation_id = operation_location_remote.split("/")[-1]

    # Call the "GET" API and wait for it to retrieve the results 
    while True:
        get_printed_text_results = computervision_client.get_read_operation_result(operation_id)
        if get_printed_text_results.status not in ['NotStarted', 'Running']:
            break
        time.sleep(1)
    text_list=[]
    # Print the detected text, line by line
    if get_printed_text_results.status == TextOperationStatusCodes.succeeded:
        for text_result in get_printed_text_results.recognition_results:
            for line in text_result.lines:
                text_list.append(line.text)
                #print(line.text)        
            
            
            
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text = text_list))        
    

# run app
if __name__ == "__main__":
    app.run(host='127.0.0.1', port=12345)

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import TextOperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import time

# Set API key.
subscription_key = 'e5847658ef394d0b960f00795989a546'
# Set endpoint.
endpoint = 'https://modulecomputervision.cognitiveservices.azure.com/'
# Call API
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

'''
Batch Read File, recognize printed text - remote
This example will extract printed text in an image, then print results, line by line.
This API call can also recognize handwriting (not shown).
'''

# Get an image with printed handwritten
remote_image_printed_text_url = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/printed_handwritten.pdf"

# Call API with URL and raw response (allows you to get the operation location)
recognize_printed_results = computervision_client.batch_read_file(remote_image_printed_text_url,  raw=True)

# Get the operation location (URL with an ID at the end) from the response
operation_location_remote = recognize_printed_results.headers["Operation-Location"]
# Grab the ID from the URL
operation_id = operation_location_remote.split("/")[-1]

# Call the "GET" API and wait for it to retrieve the results 
while True:
    get_printed_text_results = computervision_client.get_read_operation_result(operation_id)
    if get_printed_text_results.status not in ['NotStarted', 'Running']:
        break
    time.sleep(1)

# Print the detected text, line by line
if get_printed_text_results.status == TextOperationStatusCodes.succeeded:
    for text_result in get_printed_text_results.recognition_results:
        for line in text_result.lines:
            print(line.text)
            